In [1]:
import pltutils as pltu
import numpy as np
from keras import layers
from keras.layers import Conv2D, Activation, Add, Flatten, Dense, Input, Reshape, MaxPooling2D
from keras.models import Model,Sequential
from keras.datasets import cifar100
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator 
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
batch_size = 32
 

#实例化对象datagen
datagen=ImageDataGenerator()   
 
#读训练集图片
train_generator = datagen.flow_from_directory(
    'e:/tensorflow/data/ImageNet/train10',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
    'e:/tensorflow/data/ImageNet/val10',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size)
 

Found 13000 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [3]:
def ResidualBlock(filters, input_shape=(28,28,1)):
    X_input = Input(input_shape)
    X_shortcut = Conv2D(filters,(1,1), padding='same')(X_input)
    X = X_input
    X = Conv2D(filters,(3,3), padding='same')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters,(3,3), padding='same')(X)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    model = Model(inputs=X_input, outputs=X)
    return model

In [41]:
def ResNet():
    X_input = Input((224, 224,3))
    X = X_input
    X = ResidualBlock(16, input_shape=(224, 224,3))(X)
    
    X = MaxPooling2D((2,2))(X)
    X = ResidualBlock(32, input_shape=(112,112,16))(X)
    
    X = MaxPooling2D((2,2))(X)
    X = ResidualBlock(64, input_shape=(56,56,32))(X)
    
    X = MaxPooling2D((2,2))(X)
    X = ResidualBlock(128, input_shape=(28,28,64))(X)
    
    X = MaxPooling2D((2,2))(X)
    X = ResidualBlock(256, input_shape=(14,14,128))(X)
    
    X = MaxPooling2D((2,2))(X)
    X = ResidualBlock(512, input_shape=(7,7,256))(X)
    
#    X = MaxPooling2D((2,2))(X)
#     X = MaxPooling2D((2,2))(X)
#     X = MaxPooling2D((2,2))(X)
#     X = MaxPooling2D((2,2))(X)
#     X = MaxPooling2D((2,2))(X)

   
    X = Flatten()(X)
    X = Dense(1000, activation='relu')(X)
    X = Dense(10,activation='softmax')(X)
    model = Model(inputs=X_input, outputs=X)
   
    #model = Sequential()
    return model

In [42]:
model = ResNet()
model.summary()

Model: "model_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_42 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
model_42 (Model)             (None, 224, 224, 16)      2832      
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
model_43 (Model)             (None, 112, 112, 32)      14432     
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
model_44 (Model)             (None, 56, 56, 64)        57536     
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 28, 28, 64)        0  

In [43]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.SGD(lr=0.0005,learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [44]:
model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', metrics=['accuracy'])

In [30]:
import os
checkpoint_path = "checkpoints/resnet-train10/checkpoint-weights_only-epoch{epoch:04d}-valacc{val_accuracy:.4f}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)
cp_callback=ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True,period=1)

checkpoints/resnet-train10


In [45]:
model.fit_generator(generator=train_generator,steps_per_epoch=13000/batch_size,epochs=50
                   ,callbacks=[cp_callback],
                   validation_data=validation_generator,validation_steps=32)

Epoch 1/50
407/406 [==============================] - 83s 204ms/step - loss: 85.5549 - accuracy: 0.2056 - val_loss: 1.8030 - val_accuracy: 0.3000

Epoch 00001: saving model to checkpoints/resnet-train10/checkpoint-weights_only-epoch0001-valacc0.3000.ckpt
Epoch 2/50
407/406 [==============================] - 81s 200ms/step - loss: 1.5950 - accuracy: 0.4070 - val_loss: 1.2663 - val_accuracy: 0.5100

Epoch 00002: saving model to checkpoints/resnet-train10/checkpoint-weights_only-epoch0002-valacc0.5100.ckpt
Epoch 3/50
407/406 [==============================] - 81s 199ms/step - loss: 1.3540 - accuracy: 0.4885 - val_loss: 1.4296 - val_accuracy: 0.5020

Epoch 00003: saving model to checkpoints/resnet-train10/checkpoint-weights_only-epoch0003-valacc0.5020.ckpt
Epoch 4/50
407/406 [==============================] - 80s 196ms/step - loss: 1.2629 - accuracy: 0.5322 - val_loss: 1.1057 - val_accuracy: 0.4960

Epoch 00004: saving model to checkpoints/resnet-train10/checkpoint-weights_only-epoch0004-v

407/406 [==============================] - 77s 189ms/step - loss: 1.4549 - accuracy: 0.4883 - val_loss: 1.2692 - val_accuracy: 0.4240

Epoch 00033: saving model to checkpoints/resnet-train10/checkpoint-weights_only-epoch0033-valacc0.4240.ckpt
Epoch 34/50
407/406 [==============================] - 78s 191ms/step - loss: 1.4150 - accuracy: 0.4899 - val_loss: 1.3060 - val_accuracy: 0.4920

Epoch 00034: saving model to checkpoints/resnet-train10/checkpoint-weights_only-epoch0034-valacc0.4920.ckpt
Epoch 35/50
407/406 [==============================] - 77s 190ms/step - loss: 1.5174 - accuracy: 0.4945 - val_loss: 1.6259 - val_accuracy: 0.4940

Epoch 00035: saving model to checkpoints/resnet-train10/checkpoint-weights_only-epoch0035-valacc0.4940.ckpt
Epoch 36/50
407/406 [==============================] - 84s 206ms/step - loss: 1.4628 - accuracy: 0.4998 - val_loss: 1.4169 - val_accuracy: 0.4680

Epoch 00036: saving model to checkpoints/resnet-train10/checkpoint-weights_only-epoch0036-valacc0.46